User Stories Classification with textCNN

In [14]:
from torch.optim import Adam
from torch.utils.data import Dataset
import pandas as pd
import re
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from nltk import download, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import torch
import torch.nn as nn
from nltk.tokenize import word_tokenize
from collections import Counter
import string
from nltk.corpus import stopwords
import torch.nn.functional as F
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
download('punkt')
download('wordnet')
download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))




#data-load
df = pd.read_csv('dataset/user_stories_data.csv')
text_data = df['texts'].tolist()
label_data = df['labels'].tolist()
labels = {'Usability':0,
          'Functional':1,
          'Maintainability':2,
          'Security':3,
          'Portability':4,
          'Performance':5,
          'Compatibility':6,
          'Reliability':7
          }
label_counts = Counter(label_data) #data distribution
encoded_label_data = [labels[label] for label in label_data]



#preprocessing
def get_voc(texts):
    words = []
    for text in texts:
        tokens = word_tokenize(text)
        tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
        words.extend(tokens)
    word_counts = Counter(words)
    vocab = ['<pad>', '<unk>'] + [word for word, freq in word_counts.items() if freq > 1]
    vocab = {word: index for index, word in enumerate(vocab)}

    return vocab


def load_stopwords(stopwords_file):
    with open(stopwords_file, 'r', encoding='utf-8') as f:
        stopwords = [line.strip() for line in f.readlines()]
    return stopwords

lemmatizer = WordNetLemmatizer()
def preprocess_texts(texts):
    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN

    def remove_unwanted_chars(text):
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        text = re.sub(r'<.*?>', '', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        return text

    def remove_stopwords(sentence):
        tokens = word_tokenize(sentence)
        filtered_sentence = [w for w in tokens if not w in stop_words]
        return ' '.join(filtered_sentence)

    def remove_punctuation(sentence):
        translator = str.maketrans('', '', string.punctuation)
        lower_case_sentence = sentence.lower().translate(translator)
        words = lower_case_sentence.split()
        return ' '.join(words)

    def lemmatize_text(text):
        tokens = word_tokenize(text)
        tagged_tokens = pos_tag(tokens)
        lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in tagged_tokens]
        return ' '.join(lemmatized_tokens)

    #remove URL and unwanted words
    texts = [remove_unwanted_chars(text) for text in texts]
    # remove stopwords and punctuation
    texts = [remove_punctuation(text) for text in texts]
    texts = [remove_stopwords(text) for text in texts]
    # lematization
    texts = [lemmatize_text(text) for text in texts]
    # get vocabulary
    voc = get_voc(texts)
    return texts,voc



#convert text to index
def convert_texts_to_index(texts,max_length):
    texts,voc = preprocess_texts(texts)
    text_indexs =[]
    for text in texts:
        unk_id = voc['<unk>']
        pad_id = voc['<pad>']
        tokens = word_tokenize(text)
        text_index = [voc.get(word,unk_id) for word in tokens]
        if len(text_index) < max_length:
            text_index += [pad_id] * (max_length - len(text_index))
        else:
            text_index = text_index[:max_length]
        text_indexs.append(text_index)
    return text_indexs


#model
class TextCNN(nn.Module):
     def __init__(self, vocab_size, embedding_dim, output_size, filter_num=100, kernel_list=(3, 4, 5), dropout=0.5):
        super(TextCNN,self).__init__()
        # embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # cnn layer and pool layer
        self.convs = nn.ModuleList(
                [nn.Conv2d(1,filter_num,(kernel,embedding_dim)) for kernel in kernel_list])
        #dropout layer
        self.dropout = nn.Dropout(dropout)
        #full connected layer
        self.fc = nn.Linear(filter_num*len(kernel_list),output_size)
     def forward(self,x):
        x = self.embedding(x)
        x = x.unsqueeze(1)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs]
        x = [F.max_pool1d(item, item.size(2)).squeeze(2) for item in x]
        x = torch.cat(x, 1)
        x = self.dropout(x)
        logits = self.fc(x)
        return logits


class TextDataSet(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.texts[idx], dtype=torch.long), torch.tensor(self.labels[idx], dtype=torch.long)

# main
maxlength = 50
learing_rate = 0.01
embedding_dim = 128
output_size = 8
voc_size = len(get_voc(text_data))
all_data = convert_texts_to_index(text_data,maxlength)
model = TextCNN(voc_size,embedding_dim,output_size)
# spilt data
train_val_data, test_data, train_val_labels, test_labels = train_test_split(
    all_data, encoded_label_data, test_size=0.2, random_state=42)
train_data, val_data, train_labels, val_labels = train_test_split(
    train_val_data, train_val_labels, test_size=0.25, random_state=42)  # 0.25*0.8 = 0.2

train_dataset = TextDataSet(train_data,train_labels)
test_dataset = TextDataSet(test_data,test_labels)
val_dataset = TextDataSet(val_data,val_labels)
batch_size = 128  

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5)

# train and validation 
epochs = 300
patience = 10
min_delta = 0.01
best_val_loss = float('inf')
best_epoch = 0

for epoch in range(epochs):
 
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for texts, labels in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
        texts, labels = texts.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted_labels = torch.max(outputs, 1)
        correct = (predicted_labels == labels).sum().item()
        total_correct += correct
        total_samples += labels.size(0)
    
    train_loss = total_loss / len(train_loader)
    train_accuracy = total_correct / total_samples
    
# validation
    model.eval()
    total_val_loss = 0
    total_correct = 0
    with torch.no_grad():
        for texts, labels in val_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
    val_loss = total_val_loss / len(val_loader)
    val_accuracy = total_correct / len(val_loader.dataset)
    
    val_loss = total_val_loss / len(val_loader)
    val_acc = val_accuracy
    print(
        f'''Epochs: {epoch + 1}
       Train Loss: {train_loss:.3f}
       Train Accuracy: {train_accuracy:.3f}
       Val Loss: {val_loss:.3f}
       Val Accuracy: {val_accuracy:.3f}'''
    )
    if best_val_loss - val_loss > min_delta:
        best_val_loss = val_loss
        best_epoch = epoch

        torch.save(model.state_dict(), 'best_model.pth')
    elif (epoch - best_epoch) >= patience:
        print(f"Stopping early at epoch {epoch + 1}. Best epoch was {best_epoch + 1} with loss {best_val_loss}.")
        break

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\38673\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\38673\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\38673\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Training Epoch 1: 100%|███████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 145.36it/s]


Epochs: 1
       Train Loss: 2.019
       Train Accuracy: 0.246
       Val Loss: 1.747
       Val Accuracy: 0.350


Training Epoch 2: 100%|███████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 261.24it/s]


Epochs: 2
       Train Loss: 1.888
       Train Accuracy: 0.303
       Val Loss: 1.699
       Val Accuracy: 0.369


Training Epoch 3: 100%|███████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 265.27it/s]


...

Epochs: 86
       Train Loss: 0.365
       Train Accuracy: 0.904
       Val Loss: 0.989
       Val Accuracy: 0.662


Training Epoch 87: 100%|██████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 231.71it/s]


Epochs: 87
       Train Loss: 0.357
       Train Accuracy: 0.908
       Val Loss: 0.990
       Val Accuracy: 0.662


Training Epoch 88: 100%|██████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 233.02it/s]

Epochs: 88
       Train Loss: 0.355
       Train Accuracy: 0.903
       Val Loss: 0.989
       Val Accuracy: 0.669
Stopping early at epoch 88. Best epoch was 78 with loss 0.9977534979581832.


In [15]:
model.eval()
total_test_loss = 0
total_correct = 0

with torch.no_grad():
    for texts, labels in tqdm(test_loader, desc="Testing"):
        texts, labels = texts.to(device), labels.to(device)  
        outputs = model(texts)
        loss = criterion(outputs, labels)
        total_test_loss += loss.item()

        _, predicted_labels = torch.max(outputs, 1)
        total_correct += (predicted_labels == labels).sum().item()

test_accuracy = total_correct / len(test_loader.dataset)
print(f"Test Accuracy: {test_accuracy}")


Testing: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 86.90it/s]

Test Accuracy: 0.6518282988871225
